In [1]:
# 脚本功能：通过EventBridge自动获取CloudTrail中资源创建的event，触发lambda给没有打Owner标签的资源实例打上Owner标签，标签键为“Owner”，标签值为event中操作用户的用户名
# v1.8 支持ElastiCache和Opensearch Domain，由于Elasticache和Opensearch Domain需要等待资源状态就绪后才能打标签，所以使用Step Functions状态机先等待1800秒再执行Lambda函数

In [2]:
# 创建EventBridge Rule
import boto3
import json

eventbridge = boto3.client('events')
event_pattern = {
        "source": [
            "aws.elasticache",
            "aws.es"
        ],
        "detail-type": ["AWS API Call via CloudTrail"],
        "detail": {
            "eventSource": [
                "elasticache.amazonaws.com",
                "es.amazonaws.com"
            ],
            "eventName": [ 
                "CreateCacheCluster", 
                "CreateReplicationGroup",
                "CreateServerlessCache",
                "CreateDomain"
            ]
        }
    }

rule_response = eventbridge.put_rule(
        Name='monitor_elasticache_and_es_creation',
        EventPattern=json.dumps(event_pattern),
        State='ENABLED',
        Description='Monitor elasticache and opensearch creation by any user'
    )

In [3]:
# Lambda Function
import boto3
import time

def lambda_handler(event, context):
    # 设置标签键
    tag_key = "Owner"
    
    # 获取操作用户的用户名
    if event["detail"]["userIdentity"]["type"]=="IAMUser":  # 操作用户是IAM User
        user = event["detail"]["userIdentity"]["userName"]
    elif event["detail"]["userIdentity"]["type"]=="Root":  # 操作用户是Root
        user = 'root'
    elif event["detail"]["userIdentity"]["type"]=="AssumedRole":  # 操作用户是IAM Role
        user = event["detail"]["userIdentity"]["sessionContext"]["sessionIssuer"]["userName"]
    
    # 获取事务类型
    event_source = event["detail"]["eventSource"]
    event_name = event["detail"]["eventName"]

    # 定义变量，默认需要打标签
    add_tag = True
    
    # 判断事务类型，并根据是否已有Owner标签给资源对应打标签
    if event_source=="elasticache.amazonaws.com":
        tags = event["detail"]["requestParameters"]["tags"]
        if any(tag['key']==tag_key for tag in tags):  #实例没有Owner标签
            add_tag = False  #不需要打owner标签
        if add_tag:
            elasticache_client = boto3.client('elasticache')
            status = "creating"
            while status=="creating":
                #获取elasticache状态，未available的状态没法打标签
                if event_name=="CreateReplicationGroup":
                    replicationgroup_id = event["detail"]["responseElements"]["replicationGroupId"]
                    replicationgroup_response = elasticache_client.describe_replication_groups(
                                                    ReplicationGroupId=replicationgroup_id
                                                )
                    status = replicationgroup_response["ReplicationGroups"][0]["Status"]
                    elasticache_arn = event["detail"]["responseElements"]["aRN"]
                elif event_name=="CreateCacheCluster":
                    cachecluster_id = event["detail"]["responseElements"]["cacheClusterId"]
                    cachecluster_response = elasticache_client.describe_cache_clusters(
                                                CacheClusterId=cachecluster_id
                                            )
                    status = cachecluster_response["CacheClusters"][0]["CacheClusterStatus"]
                    elasticache_arn = event["detail"]["responseElements"]["aRN"]
                elif event_name=="CreateServerlessCache":
                    serverlesscache_name = event["detail"]["responseElements"]["serverlessCache"]["serverlessCacheName"]
                    serverlesscache_response = elasticache_client.describe_serverless_caches(
                                                    ServerlessCacheName=serverlesscache_name
                                               )
                    status = serverlesscache_response["ServerlessCaches"][0]["Status"]
                    elasticache_arn = event["detail"]["responseElements"]["serverlessCache"]["aRN"]
                time.sleep(10)
            if status=="available":
                elasticache_client.add_tags_to_resource(
                    ResourceName=elasticache_arn,
                    Tags=[
                        {
                            'Key': tag_key,
                            'Value': user
                        },
                    ]
                )
    elif event_source=="es.amazonaws.com":
        if event_name=="CreateDomain":
            if "tagList" in event["detail"]["requestParameters"]:
                tags = event["detail"]["requestParameters"]["tagList"]
                if any(tag['key']==tag_key for tag in tags):  #实例没有Owner标签
                    add_tag = False  #不需要打owner标签
            if add_tag:
                opensearch_arn = event["detail"]["responseElements"]["domainStatus"]["aRN"]
                opensearch_client = boto3.client('opensearch')
                opensearch_client.add_tags(
                    ARN=opensearch_arn,
                    TagList=[
                        {
                            'Key': tag_key,
                            'Value': user
                        },
                    ]
                )